In [85]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [91]:
def get_soup(id):
  url = 'https://www.law.com/law-firm-profile/?id='+str(id)
  response = requests.get(url)
  html = response.text
  soup = bs(html, 'html.parser')
  return soup

In [92]:
def get_dict(soup):
  data_dict = {}
  data_dict['Name'] = soup.find('h1', {'class':'page-title left'}).text

  box = soup.find_all('div', {'class':'col-lg-6 col-md-12 col-sm-12 module'})
  inner = box[0].find('div', {'class':'inner'})

  data_dict['Description'] = inner.find_all('p', {'class':'firms-para'})[0].text.replace('\xa0',' ').strip()
  data_dict['Website'] = inner.find('a')['href']

  inner = box[2].find('div', {'class':'inner'})
  ul = inner.find('ul', {'class':'overview mobile-overview'})
  lis = ul.find_all('li')

  for item in lis:
    key_value = item.find_all('div', {'class':'col-md-6'})
    title = key_value[0].text.strip().replace(':','').replace('*','')
    value = key_value[1].text.strip()
    
    data_dict[title] = value

  rankings = box[1].find('div', {'class':'ranking-logos'}).find_all('div', {'class':'rankings'})
  for rank in rankings:
    name = rank.find('div', {'class':'col-lg-3 col-md-12 col-sm-12'}).text.strip().replace(' ', '')
    years = rank.find('div', {'class':'col-lg-7 col-md-12 col-sm-12'}).find_all('div', {'class':'col-md-4 iphone-logos'})
    for y in years:
      year = y.find('p', {'class':'date-firms'}).text.strip()
      nameyear = name+'_'+year
      yrank = y.find('p', {'class':'rank-firms'}).text.strip().replace('#','')    
      data_dict[nameyear] = yrank

  return data_dict

In [ ]:
master_list = []
for i in range(1, 200):  
  if i>20: break  
  print(i)
  s = get_soup(i)
  try:
    d = get_dict(s)
    master_list.append(d)
  except:
    print(i, "doesn't exist")
  
master_list

In [111]:
df = pd.DataFrame(master_list)
df

,Name,Description,Website,Global Rank,Total Offices,Total Headcount,Equity Partners,Non-Equity Partners,Associates,Total Revenue,Profit Per Equity Partner,Revenue Per Lawyer,Global200_2020,Global200_2019,Global200_2018,AmLaw200_2021,AmLaw200_2020,AmLaw200_2019,NLJ500_2020,NLJ500_2019,NLJ500_2018,UKTop100_2020,UKTop100_2019,UKTop100_2018
0,Adams and Reese LLP,According to the National Law Journal's 2020 N...,http://www.adamsandreese.com,N/A,15,244,95,48,50,"$129,486,000","$592,000","$544,000",N/A,N/A,N/A,185,176,168,174,164,160,N/A,N/A,N/A
1,Addleshaw Goddard,"With $367,080,000 gross revenue in 2019, the f...",http://www.addleshawgoddard.com,136,11,893,113,0,0,"$367,080,000","$881,000","$411,000",136,131,140,N/A,N/A,N/A,N/A,N/A,N/A,22,21,20
2,Akerman,According to the National Law Journal's 2020 N...,http://www.akerman.com,112,24,666,254,150,210,"$451,682,000","$748,000","$699,000",112,121,121,86,88,94,71,73,70,N/A,N/A,N/A
3,Akin Gump Strauss Hauer & Feld LLP,According to the National Law Journal's 2020 N...,http://www.akingump.com,42,20,910,179,146,499,"$1,208,738,000","$3,023,000","$1,346,000",42,39,37,33,34,30,42,44,42,N/A,N/A,N/A
4,Allen & Overy,"With $2,160,729,000 gross revenue in 2019, the...",http://www.allenovery.com,12,38,"2,447",397,0,0,"$2,160,729,000","$2,081,000","$883,000",12,10,10,N/A,N/A,N/A,N/A,N/A,N/A,4,4,4
5,Allens,"With $320,897,000 gross revenue in 2019, the f...",http://www.aar.com.au/,151,0,726,0,0,0,"$442,000",$0,"$442,000",151,141,80,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
6,Alston Bird,According to the National Law Journal's 2020 N...,http://www.alston.com,61,11,796,153,211,370,"$851,601,000","$2,434,000","$1,094,000",61,64,56,53,51,52,58,53,48,N/A,N/A,N/A
7,Andrews Kurth,,http://www.andrewskurth.com,N/A,10,0,0,0,0,$0,$0,$0,N/A,N/A,154,N/A,N/A,N/A,N/A,N/A,143,N/A,N/A,N/A
8,Arent Fox,According to the National Law Journal's 2020 N...,http://www.arentfox.com/,148,5,385,96,85,145,"$333,600,000","$1,115,000","$866,000",148,162,146,107,108,115,116,130,123,N/A,N/A,N/A
9,Armstrong Teasdale,According to the National Law Journal's 2020 N...,http://www.armstrongteasdale.com/,N/A,9,253,75,59,100,"$149,239,000","$730,000","$567,000",N/A,N/A,N/A,174,177,180,170,181,183,N/A,N/A,N/A


In [107]:
cd 'drive/MyDrive/Web Scraper'
ls

/content/drive/MyDrive/Web Scraper


In [112]:
df.to_csv('sample_lawfirms_output.csv', index=False)